# MSSIM

# こうです

scikit-imageにあるstructural_similarityを使う  
https://kite.com/python/docs/skimage.measure.compare_ssim  
gaussian_weight = Trueにすることで、sigma=1.5のガウス関数が適用される  
アルゴリズムに用いるK1,K2はK1=0.01,K2=0.02とする
windowサイズはsigmaに依存するらしい

In [ ]:
from skimage.metrics import structural_similarity as ssim
ssim(IMG.img,Q_recover.astype(IMG.img.dtype),gaussian_weights=True,sigma=1.5,K1=0.01,K2=0.03)

https://github.com/4og/mssim より

In [1]:
class MSSIM:
    def gaussian(self, size, sigma):
        x = numpy.arange(0, size, 1, float)
        y = x[:, numpy.newaxis]
        xc = (size - 1) / 2
        yc = (size - 1) / 2

        gauss = numpy.exp(-((x - xc) ** 2 + (y - yc) ** 2) / (2 * sigma ** 2))
        return gauss / gauss.sum()

    def compute(self, im1, im2, k=[0.01, 0.03]):
        if im1.shape != im2.shape:
            print("Incorrect image. All images " "should be of equal size")
            
        c1 = (k[0] * 255) ** 2
        c2 = (k[1] * 255) ** 2
        win = self.gaussian(11, 1.5)

#         mu1 = filters.correlate(im1, win) / im1.shape[0] / im1.shape[1]
        mu1 = filters.correlate(im1, win)
        mu1_sq = mu1 * mu1
#         s1sq = filters.correlate(im1 * im1, win) - mu1_sq
        s1sq = filters.correlate((im1 - mu1)**2, win)

#         mu2 = filters.correlate(im2, win) / im2.shape[0] / im2.shape[1]
        mu2_sq = mu2 * mu2
        mu1_mu2 = mu1 * mu2

        s2sq = filters.correlate(im2 * im2, win) - mu2_sq
#         s2sq = filters.correlate((im2 - mu2)**2, win)
        s12 = filters.correlate(im1 * im2, win) - mu1_mu2

        ssims = ((2 * mu1_mu2 + c1) * (2 * s12 + c2)) / (
            (mu1_sq + mu2_sq + c1) * (s1sq + s2sq + c2)
        )
        
        return ssims.mean()

https://dftalk.jp/?p=18111

In [2]:
import sys, os, numpy, scipy.misc
from scipy.ndimage import filters
from tqdm._tqdm_notebook import tqdm_notebook


class MSSIM:
    def gaussian(self, size, sigma):
        x = numpy.arange(0, size, 1, float)
        y = x[:, numpy.newaxis]
        xc = (size - 1) / 2
        yc = (size - 1) / 2

        gauss = numpy.exp(-((x - xc) ** 2 + (y - yc) ** 2) / (2 * sigma ** 2))
        return gauss / gauss.sum()

    def mirror(self, x, x_min, x_max):
        while x < x_min or x >= x_max:
            if x < x_min:
                x = x_min + (x_min - x - 1)
            if x >= x_max:
                x = x_max + (x_max - x - 1)
        return x

    def compute(self, im1, im2, k=[0.01, 0.03]):
        if im1.shape != im2.shape:
            print("Incorrect image. All images " "should be of equal size")
            return

        height = im1.shape[0]
        width = im1.shape[1]
        c1 = (k[0] * 255) ** 2
        c2 = (k[1] * 255) ** 2
        win = self.gaussian(11, 1.5)

        # 円対象ガウス関数による畳込み　重み付けを全体に適用
        # filters.correlate は defaultで mode='reflect'
        # 窓サイズが11x11なので こんな感じに反転 43210 012345
        m = (11 - 1) // 2
        ssim_sum = 0
        for y in tqdm_notebook(range(height)):
            for x in range(width):
                ave1 = ave2 = 0  # 平均
                var1 = var2 = 0  # 分散
                cov = 0  # 共分散
                arr1 = np.zeros((11, 11))
                arr2 = np.zeros((11, 11))
                arr3 = np.zeros((11, 11))

                for dy in range(-m, m + 1):
                    for dx in range(-m, m + 1):
                        mx = self.mirror(x + dx, 0, width)
                        my = self.mirror(y + dy, 0, height)

                        #                         ave1 += im1[mx, my] * win[5 + dx, 5 + dy]
                        #                         ave1 += im1[mx, my]
                        arr1[5 + dx, 5 + dy] = im1[mx, my]
                        #                         ave2 += im2[mx, my] * win[5 + dx, 5 + dy]
                        #                         ave2 += im2[mx, my]
                        arr2[5 + dx, 5 + dy] = im2[mx, my]

                #                 ave1 /= (m * 2 + 1) ** 2
                ave1 = np.sum(np.sum(arr1)) / (m * 2 + 1) ** 2
                #                 ave2 /= (m * 2 + 1) ** 2
                ave2 = np.sum(np.sum(arr2)) / (m * 2 + 1) ** 2

                for dy in range(-m, m + 1):
                    for dx in range(-m, m + 1):
                        mx = self.mirror(x + dx, 0, width)
                        my = self.mirror(y + dy, 0, height)

                        #                         var1 += (im1[mx, my] - ave1) ** 2 * win[5+dx, 5+dy]
                        #                         var1 += (im1[mx, my] - ave1) ** 2
                        arr1[5 + dx, 5 + dy] = im1[mx, my] - ave1

                        #                         var2 += (im2[mx, my] - ave2) ** 2 * win[5+dx, 5+dy]
                        #                         var2 += (im2[mx, my] - ave2) ** 2
                        arr2[5 + dx, 5 + dy] = im2[mx, my] - ave2
                #                         cov += (im1[mx, my] - ave1) * (im1[mx, my] - ave2) * win[5+dx,5+dy]
                #                         cov += (im1[mx, my] - ave1) * (im1[mx, my] - ave2)

                arr3 = arr1 * arr2

                #                 var1 /= (m * 2 + 1) ** 2
                var1 = np.np.sum(np.sum(arr1)) / (m * 2 + 1) ** 2
                #                 var2 /= (m * 2 + 1) ** 2
                var2 = np.sum(np.sum(arr2)) / (m * 2 + 1) ** 2
                #                 cov /= (m * 2 + 1) ** 2
                cov = np.sum(np.sum(arr3)) / (m * 2 + 1) ** 2

                ssim_sum += ((2 * ave1 * ave2 + c1) * (2 * cov + c2)) / (
                    (ave1 ** 2 + ave2 ** 2 + c1) * (var1 + var2 + c2)
                )

        return ssim_sum / (width * height)

In [4]:
from tqdm._tqdm_notebook import tqdm_notebook
from skimage.metrics import structural_similarity as ssim


class MSSIM:
    def mirror(self, x, x_min, x_max):
        while x < x_min or x >= x_max:
            if x < x_min:
                x = x_min + (x_min - x - 1)
            if x >= x_max:
                x = x_max + (x_max - x - 1)
        return x

    def compute(self, im1, im2):
        if im1.shape != im2.shape:
            print("Incorrect image. All images " "should be of equal size")
            return

        height = im1.shape[0]
        width = im1.shape[1]
        # 円対象ガウス関数による畳込み　重み付けを全体に適用
        # filters.correlate は defaultで mode='reflect'
        # 窓サイズが11x11なので こんな感じに反転 43210 012345
        m = (11 - 1) // 2
        ssim_sum = 0
        for y in tqdm_notebook(range(height)):
            for x in range(width):
                arr1 = np.zeros((11, 11))
                arr2 = np.zeros((11, 11))

                for dy in range(-m, m + 1):
                    for dx in range(-m, m + 1):
                        mx = self.mirror(x + dx, 0, width)
                        my = self.mirror(y + dy, 0, height)

                        arr1[5 + dx, 5 + dy] = im1[mx, my]
                        arr2[5 + dx, 5 + dy] = im2[mx, my]

                ssim_sum +=ssim(arr1,arr2,gaussian_weights=True,K1=0.01,K2=0.03)

        return ssim_sum / (width * height)

In [ ]:
# Usage: mssim.py reference-image other-images
MSSIM().compute(IMG.img, Q_recover)

ssimの画像を出せるらしい

In [ ]:
m = 5
arr1 = np.zeros((11,11))
arr2 = np.zeros((11,11))

im1 = IMG.img
im2 = Q_recover
width = im1.shape[0]
height = im1.shape[1]

x = 0
y = 0

for dy in range(-m, m + 1):
    for dx in range(-m, m + 1):
        mx = MSSIM().mirror(x + dx, 0, width)
        my = MSSIM().mirror(y + dy, 0, height)

        arr1[5 + dx, 5 + dy] = im1[mx, my]
        arr2[5 + dx, 5 + dy] = im2[mx, my]

plt.imshow(ssim(arr1,arr2,full=True,gaussian_weights=True,K1=0.01,K2=0.03)[1])